
# Rust (1987) Problem Set


## Initialization
- preliminaries - 
- load data - 
- set fixed vals -
- starting vals - 

## Optimization 
- <- Relies on LL( ) function!

- LL() <- relies on ContractionMap Function!
    - Sets case for starting iterations (matrix of 0)
        - runs cmap() on this base case
    - Sets case for iteration greater than 0
        - runs cmap() on previous output etc...
    - after cmap() is complete, we can use the prob equation 
      to find choice probabilities
    - outputs LL (product of choice probabilities)
    - outputs ll (-log(LL))
        
## ContractionMap( ) - 

- set v0 (first column of v (age))
- set v1 (second col of v (choice))
- theta is fixed
- R is fixed
- tol is fixed
- dif is fixed at 1 for first iteration

- while loop: 
    - update value vector for not replace choice
    - update value vector for replace chouce
    - computer difference 
    - iteration??


- **Returns a contraction map from intiial values**




            
            
        
        
    
    




In [1]:
# use pandas library for data manipulation
import pandas as pd
import numpy as np
import math as m
import scipy.optimize as sp

fname = "PSet2_data.csv"

# load dataset and name columns
df = pd.read_csv(fname, names = ("age","choice"))

# probably not necessary, but I subset the df for each column 
age = df["age"]
choice = df["choice"]

# set initial values
beta = .9
dim = 5
N = len(df["age"])
gamma = .5775
iteration = 0 


# Parameters suggestions for ML procedure:
theta0 = -1 
R0 = -2

# create list of params
param0 = [theta0, R0]
N_p = len(param0 )
size = (dim,2)

# initialize a vector of zeros
v = np.zeros(size)

# not sure if this is necessary??
iteration =0



In [2]:
def contract_map(v, parameters_ml):

    iteration = 0
    vv0 = np.zeros((5,1))
    vv1 = np.zeros((5,1)) 
    
    v0 = np.array([v[:,0]]).T
    v1 = np.array([v[:,1]]).T
    
    theta = parameters_ml[0]
    R = parameters_ml[1]
    tolerance = 10**(-30)
    diff = 1
    

    
    while diff > tolerance:
        
        vv0 = np.zeros((5,1))
        vv1 = np.zeros((5,1)) 
        

        vv0[0] = theta*1+ beta*(gamma+m.log((m.exp(v0[1])+m.exp(v1[1]))))
        vv0[1] = theta*2+ beta*(gamma+m.log((m.exp(v0[2])+m.exp(v1[2]))))
        vv0[2] = theta*3+ beta*(gamma+m.log((m.exp(v0[3])+m.exp(v1[3]))))
        vv0[3] = theta*4+ beta*(gamma+m.log((m.exp(v0[4])+m.exp(v1[4]))))
        vv0[4] = theta*5+ beta*(gamma+m.log((m.exp(v0[4])+m.exp(v1[4]))))


        vv1[0] = R + beta*(gamma+m.log((m.exp(v0[0])+m.exp(v1[0]))))
        vv1[1] = R + beta*(gamma+m.log((m.exp(v0[0])+m.exp(v1[0]))))
        vv1[2] = R + beta*(gamma+m.log((m.exp(v0[0])+m.exp(v1[0]))))
        vv1[3] = R + beta*(gamma+m.log((m.exp(v0[0])+m.exp(v1[0]))))
        vv1[4] = R + beta*(gamma+m.log((m.exp(v0[0])+m.exp(v1[0]))))



        diff1 = max(abs(vv1 - v1))
        diff0 = max(abs(vv0 - v0))
        diff = max(diff1, diff0)

        
        v0 = vv0
        v1 = vv1

        iteration = iteration + 1
        
    return([v0,v1])

    
    
    

        

In [3]:
def LogLik(parameters_m1):
    cp = np.zeros((N,1))
    
    size = (dim,2)
    v = np.zeros(size)
    VV = np.zeros(size)

    
    
    if iteration == 0:

        VV = contract_map(v, parameters_m1)
        
        choice_p0 = np.multiply(np.exp(VV[0]), (np.exp(VV[0])+np.exp(VV[1]))**-1)
        choice_p1 = np.multiply(np.exp(VV[1]), (np.exp(VV[0])+np.exp(VV[1]))**-1)
    
    else:
        v = VV
        VV = contract_map(v, parameters_m1)
        
        
        choice_p0 = np.multiply(np.exp(VV[0]), (np.exp(VV[0])+np.exp(VV[1]))**-1)
        choice_p1 = np.multiply(np.exp(VV[1]), (np.exp(VV[0])+np.exp(VV[1]))**-1)
    

    # I'd like to clean this section up... it feels like too many logical statements
    for i in range(0,N):
        
        if df["choice"][i] == 0 and df["age"][i] == 1:
            
            cp[i] = choice_p0[0]

        elif df["choice"][i] == 0 and df["age"][i] == 2:

            cp[i] = choice_p0[1]

        elif df["choice"][i] == 0 and df["age"][i] == 3:

            cp[i] = choice_p0[2]

        elif df["choice"][i] == 0 and df["age"][i] == 4:

            cp[i] = choice_p0[3]

        elif df["choice"][i] == 0 and df["age"][i] == 5:

            cp[i] = choice_p0[4]
        
        elif df["choice"][i] == 1 and df["age"][i] == 1:
            
            cp[i] = choice_p1[0]

        elif df["choice"][i] == 1 and df["age"][i] == 2:

            cp[i] = choice_p1[1]

        elif df["choice"][i] == 1 and df["age"][i] == 3:

            cp[i] = choice_p1[2]

        elif df["choice"][i] == 1 and df["age"][i] == 4:

            cp[i] = choice_p1[3]

        elif df["choice"][i] == 1 and df["age"][i] == 5:

            cp[i] = choice_p1[4]
                
              
    # log likelihood            
    LL = np.prod(cp)
    
    # negative LL for fmin procedure
    ll = -m.log(LL)
    return(ll)
    


In [4]:

# run algorithm and print parameters (use fmin...Not worried about speed at the moment)

params = sp.fmin(LogLik, param0)

print(params)

Optimization terminated successfully.
         Current function value: 424.158893
         Iterations: 46
         Function evaluations: 91
[-1.14839668 -4.44645146]


# Question 3: 

For what value of $\epsilon_{0,t} - \epsilon_{1,t}$ is the firm indifferent between replacing its machine or not?



Note that: 

$$
\Pi(a_t,i_t,\epsilon_{0,t}, \epsilon_{1,t}) = \begin{cases} 
      \theta_1 a_t +\epsilon_{0,t} & \text{if } i_t = 0 \\
      R +\epsilon_{1,t}  & \text{if } i_t = 1 \\
   \end{cases}
$$

Suppose $a_t = 2$, then:

$$
\Pi(a_t,i_t,\epsilon_{0,t}, \epsilon_{1,t}) = \begin{cases} 
      -1.14838832 * 2 +\epsilon_{0,t} & \text{if } i_t = 0 \\
      -4.44640774 +\epsilon_{1,t}  & \text{if } i_t = 1 \\
   \end{cases}
$$


So

$$-1.14838832 * 2 + \epsilon_{0,t} =  -4.44640774 +\epsilon_{1,t} \\
\epsilon_{0,t} - \epsilon_{1,t} = -4.44640774  + (1.14838832 * 2)
$$


In [5]:
indif = -1*(opt[0] *2)  +  opt[1]
print(indif)

print(" --------------- \n probability of choosing to replace machine?")
print(max(opt[0]*2, opt[1]))

print(" ------------- \n probability is 0?")

a = opt[0]*4 +1
b = opt[1] - 1.5
print(" -------------")
print(max(a,b))

print("option to not replace is chosen")







-2.1496311021531023
 --------------- 
 probability of choosing to replace machine?
-2.2967766354829724
 ------------- 
 probability is 0?
 -------------
-3.593553270965945
option to not replace is chosen


In [5]:
# parmas = np.random.normal(-4,5,(10,2))
# parmas = parmas.tolist()


# param0
# parmas = [[-1,-2],[-1.5,-2.5], [-.4,-1],[-.6,-2],[-.2,-1]]


# a = []
# for i in range(0,5):
#    a.append(sp.fmin(LogLik, parmas[i]))



